<a href="https://colab.research.google.com/github/yanzhen4/feedback-prize-english-language-learning/blob/master/DeBERTa-pseudoLabel_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About this notebook
- Deberta-v3-base starter code
- pip wheels is [here](https://www.kaggle.com/code/yasufuminakama/fb3-pip-wheels)
- Inference notebook is [here](https://www.kaggle.com/yasufuminakama/fb3-deberta-v3-base-baseline-inference)

If this notebook is helpful, feel free to upvote :)

Package: pip install wandb transformers tokenizers iterative-stratification

In [2]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = '/content/gdrive/MyDrive/Colab_Notebooks/Deberta/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Directory settings

In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


# CFG

In [4]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=True
    competition='FB3'
    _wandb_kernel='nakama'
    debug=False
    apex=True
    print_freq=20
    num_workers=4
    model="/content/gdrive/MyDrive/Colab_Notebooks/Deberta/microsoft_deberta-large"
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=5
    encoder_lr=2e-6
    decoder_lr=2e-6
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [5]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    
    import wandb

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret_value_0 = user_secrets.get_secret("wandb_api")
        wandb.login(key=secret_value_0)
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='FB3-Public', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Library

In [6]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

#os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

#os.system('pip uninstall -y transformers')
#os.system('pip uninstall -y tokenizers')
#os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels transformers')
#os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels tokenizers')
#import tokenizers
import transformers
#print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transformers.__version__: 4.24.0
env: TOKENIZERS_PARALLELISM=true


# Utils

In [7]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds): #loss
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores #score: single prediction result; scores: list of all results


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'train'): #print log 
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# Data Loading

In [8]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('/content/gdrive/MyDrive/Colab_Notebooks/Deberta/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/Colab_Notebooks/Deberta/test.csv')
submission = pd.read_csv('/content/gdrive/MyDrive/Colab_Notebooks/Deberta/sample_submission.csv')
old_train = pd.read_csv("/content/gdrive/MyDrive/Colab_Notebooks/Deberta/old_train.csv")

In [9]:
#Test
train = train.head(1000)
print(f"train.shape: {train.shape}")
display(train.head())
print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

train.shape: (1000, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [10]:
size = 200
old_train = old_train[["discourse_text"]].rename(columns={"discourse_text": "full_text"})
old_train = old_train[old_train["full_text"].str.len() >= 500].head(size).reset_index()
old_train = old_train.drop(columns = ['index'])
initial_values = np.zeros((len(old_train), 6))
old_train[CFG.target_cols] = initial_values
old_train

,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,According to an article by the Edgar Snyder Fi...,0.0,0.0,0.0,0.0,0.0,0.0
1,The most common of these distractions is a cel...,0.0,0.0,0.0,0.0,0.0,0.0
2,Reaction time is the measure of how quickly an...,0.0,0.0,0.0,0.0,0.0,0.0
3,"The affects can be physical, emotional, or psy...",0.0,0.0,0.0,0.0,0.0,0.0
4,"In conclusion, people shouldn't use cellphones...",0.0,0.0,0.0,0.0,0.0,0.0
5,It could lead to accidents and altercations. Y...,0.0,0.0,0.0,0.0,0.0,0.0
6,Many people happen to pass away due to a motor...,0.0,0.0,0.0,0.0,0.0,0.0
7,"In conclusion, Drivers shouldn't drive while u...",0.0,0.0,0.0,0.0,0.0,0.0
8,Other drivers take notice when drivers are on ...,0.0,0.0,0.0,0.0,0.0,0.0
9,Driving with cell phones should be banned beca...,0.0,0.0,0.0,0.0,0.0,0.0


# CV split

In [11]:
# ====================================================
# CV split
# ====================================================
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    250
1    250
2    250
3    250
dtype: int64

In [12]:
len(train)

1000

In [13]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

# tokenizer

In [14]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model) #get deberta tokenizer 
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

# Dataset

In [15]:
# ====================================================
# Define max_len
# Find max token length after tokenizing each text 
# ====================================================
lengths = []
tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 3 # cls & sep & sep
LOGGER.info(f"max_len: {CFG.max_len}")
CFG.max_len

  0%|          | 0/1000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors
max_len: 4182
INFO:__main__:max_len: 4182


4182

In [16]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs #a list of tensors 


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label
    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

# Model

In [17]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float() 
        #last_hidden_state is a list of n tensors, n is sequence length
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        #padding: [PAD] make same length, for [PAD] attention_mask = 0
        #last_hidden_state is output of the model
        return mean_embeddings
    

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True) 
            #output of every layer during inference
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 6) #hidden_size is input size, 6 is output size 
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

# Loss

In [18]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

# Helpler functions

In [19]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        #optimize
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs) #predict
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

# train loop

In [20]:
#testing
# ====================================================
# train loop
# ====================================================
import math
def train_loop(folds, fold): 
     
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    # num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    # scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    pesudoLabel_data = old_train #dataframe
    pesudoLabel_data_len = len(pesudoLabel_data)
    num_sample = 5
    sample_size = int(pesudoLabel_data_len / num_sample)

    num_train_steps = int((len(train_folds) + math.ceil(sample_size * num_sample / 2)) / CFG.batch_size * CFG.epochs * num_sample + len(train_folds) 
                          / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean') # RMSELoss(reduction="mean")
    
    best_score = np.inf
    lr_scaler = 4
    # 5 Epoch with origin data
    for epoch in range(CFG.epochs):

        start_time = time.time()
         
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    #unlabled_data = old data
    
    for i in range(1, 6):
        # new scheduler
        # optimizer_parameters = get_optimizer_params(model,
        #                                         encoder_lr=CFG.encoder_lr / lr_scaler, 
        #                                         decoder_lr=CFG.decoder_lr / lr_scaler,
        #                                         weight_decay=CFG.weight_decay)
        # optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr / lr_scaler, eps=CFG.eps, betas=CFG.betas)
        # lr_scaler = lr_scaler * lr_scaler
        
        # num_train_steps = int((len(train_folds) + i * sample_size) / CFG.batch_size * CFG.epochs)
        # scheduler = get_scheduler(CFG, optimizer, num_train_steps)

        pesudoLabel_data_use = pesudoLabel_data[:sample_size]
        pesudoLabel_data = pesudoLabel_data[sample_size:]
        #unlabled_data = unlabled_data - sampled_data
        pesudo_dataset = TrainDataset(CFG, pesudoLabel_data_use)
        pesudo_loader = DataLoader(pesudo_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
        preds = np.array([[]])
        start = end = time.time()
        i = 0
        for step, (inputs, labels) in enumerate(pesudo_loader):
            inputs = collate(inputs)
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            with torch.no_grad():
                y_preds = model(inputs) #predict
            if i != 0:
              preds = np.concatenate((preds, y_preds.to('cpu').numpy()), axis = 0)
            else:
              i = 1
              preds = y_preds.to('cpu').numpy()
            end = time.time()
        
        print(np.array(preds).shape, pesudoLabel_data_use.columns)
        pesudoLabel_data_use[CFG.target_cols] = preds
        print(train_folds.columns)
        print(pesudoLabel_data_use.columns)
        frames = [train_folds, pesudoLabel_data_use]
        train_folds = pd.concat(frames)
        train_dataset = TrainDataset(CFG, train_folds)
        train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)

        # 5 Epochs training with origin data + pseudo-labled data
        for epoch in range(CFG.epochs):

            
                
            start_time = time.time()

            #predict for all sampled_data, generate labled data
            #stack sampled_data on new data
            #add to train_loader
            
            # train
            avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

            # eval
            avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
            
            # scoring
            score, scores = get_score(valid_labels, predictions)

            elapsed = time.time() - start_time

            LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
            LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
            if CFG.wandb:
                wandb.log({f"[fold{fold}] epoch": epoch+1, 
                          f"[fold{fold}] avg_train_loss": avg_loss, 
                          f"[fold{fold}] avg_val_loss": avg_val_loss,
                          f"[fold{fold}] score": score})
            
            if best_score > score:
                best_score = score
                LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
                torch.save({'model': model.state_dict(),
                            'predictions': predictions},
                            OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            
    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [22]:
#testing
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
                break
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
    if CFG.wandb:
        wandb.finish()

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
DebertaConfig {
  "_name_or_path": "/content/gdrive/MyDrive/Colab_Notebooks/Deberta/microsoft_deberta-large",
  "architectures": [
    "DebertaModel"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "c2p",
    "p2c"
  ],
  "position_biased_input": false,
  "relative_attention": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 0,
  "vocab_size":

Epoch: [1][0/750] Elapsed 0m 4s (remain 57m 15s) Loss: 2.7481(2.7481) Grad: inf  LR: 0.00000200  
Epoch: [1][20/750] Elapsed 0m 10s (remain 5m 54s) Loss: 1.8999(2.1047) Grad: 382233.0312  LR: 0.00000200  
Epoch: [1][40/750] Elapsed 0m 15s (remain 4m 30s) Loss: 2.3224(1.8411) Grad: 456535.5000  LR: 0.00000200  
Epoch: [1][60/750] Elapsed 0m 21s (remain 3m 58s) Loss: 0.2473(1.4236) Grad: 405036.4062  LR: 0.00000200  
Epoch: [1][80/750] Elapsed 0m 26s (remain 3m 40s) Loss: 0.0941(1.1234) Grad: 234005.9844  LR: 0.00000200  
Epoch: [1][100/750] Elapsed 0m 32s (remain 3m 26s) Loss: 0.9347(0.9415) Grad: 579065.8125  LR: 0.00000200  
Epoch: [1][120/750] Elapsed 0m 38s (remain 3m 19s) Loss: 0.0800(0.8231) Grad: 230424.7031  LR: 0.00000200  
Epoch: [1][140/750] Elapsed 0m 44s (remain 3m 10s) Loss: 0.1007(0.7336) Grad: 250333.1719  LR: 0.00000200  
Epoch: [1][160/750] Elapsed 0m 50s (remain 3m 3s) Loss: 0.0546(0.6639) Grad: 176067.3750  LR: 0.00000200  
Epoch: [1][180/750] Elapsed 0m 55s (remain 

Epoch 1 - avg_train_loss: 0.2690  avg_val_loss: 0.1742  time: 231s
INFO:__main__:Epoch 1 - avg_train_loss: 0.2690  avg_val_loss: 0.1742  time: 231s
Epoch 1 - Score: 0.5986  Scores: [0.641151946503038, 0.5158336806909397, 0.562247368306175, 0.6336507957886203, 0.620643569047055, 0.6182538218846939]
INFO:__main__:Epoch 1 - Score: 0.5986  Scores: [0.641151946503038, 0.5158336806909397, 0.562247368306175, 0.6336507957886203, 0.620643569047055, 0.6182538218846939]
Epoch 1 - Save Best Score: 0.5986 Model
INFO:__main__:Epoch 1 - Save Best Score: 0.5986 Model


EVAL: [124/125] Elapsed 0m 15s (remain 0m 0s) Loss: 0.1423(0.1742) 
Epoch: [2][0/750] Elapsed 0m 0s (remain 5m 31s) Loss: 0.0951(0.0951) Grad: inf  LR: 0.00000200  
Epoch: [2][20/750] Elapsed 0m 5s (remain 3m 24s) Loss: 0.0512(0.1130) Grad: 159013.7344  LR: 0.00000200  
Epoch: [2][40/750] Elapsed 0m 12s (remain 3m 30s) Loss: 0.1020(0.1009) Grad: 265847.5000  LR: 0.00000200  
Epoch: [2][60/750] Elapsed 0m 18s (remain 3m 26s) Loss: 0.1301(0.0933) Grad: 257116.5312  LR: 0.00000199  
Epoch: [2][80/750] Elapsed 0m 23s (remain 3m 16s) Loss: 0.3389(0.0974) Grad: 569955.7500  LR: 0.00000199  
Epoch: [2][100/750] Elapsed 0m 29s (remain 3m 8s) Loss: 0.1435(0.0945) Grad: 319821.9375  LR: 0.00000199  
Epoch: [2][120/750] Elapsed 0m 34s (remain 3m 1s) Loss: 0.1142(0.0919) Grad: 311512.4062  LR: 0.00000199  
Epoch: [2][140/750] Elapsed 0m 40s (remain 2m 54s) Loss: 0.0650(0.1050) Grad: 187572.8281  LR: 0.00000199  
Epoch: [2][160/750] Elapsed 0m 45s (remain 2m 48s) Loss: 0.0420(0.1055) Grad: 183787.8

Epoch 2 - avg_train_loss: 0.1089  avg_val_loss: 0.1118  time: 227s
INFO:__main__:Epoch 2 - avg_train_loss: 0.1089  avg_val_loss: 0.1118  time: 227s
Epoch 2 - Score: 0.4744  Scores: [0.4955053859707873, 0.48258737323618073, 0.43359696928816804, 0.4744697259056485, 0.4935083034115342, 0.4669559239043084]
INFO:__main__:Epoch 2 - Score: 0.4744  Scores: [0.4955053859707873, 0.48258737323618073, 0.43359696928816804, 0.4744697259056485, 0.4935083034115342, 0.4669559239043084]
Epoch 2 - Save Best Score: 0.4744 Model
INFO:__main__:Epoch 2 - Save Best Score: 0.4744 Model


...

EVAL: [124/125] Elapsed 0m 15s (remain 0m 0s) Loss: 0.0854(0.1277) 
Epoch: [5][0/950] Elapsed 0m 0s (remain 8m 4s) Loss: 0.0000(0.0000) Grad: 1990.8517  LR: 0.00000000  
Epoch: [5][20/950] Elapsed 0m 6s (remain 4m 28s) Loss: 0.0000(0.0003) Grad: 6170.4355  LR: 0.00000000  
Epoch: [5][40/950] Elapsed 0m 11s (remain 4m 16s) Loss: 0.0000(0.0002) Grad: 3946.4648  LR: 0.00000000  
Epoch: [5][60/950] Elapsed 0m 17s (remain 4m 8s) Loss: 0.0000(0.0002) Grad: 5246.5679  LR: 0.00000000  
Epoch: [5][80/950] Elapsed 0m 22s (remain 4m 1s) Loss: 0.0000(0.0002) Grad: 4534.0371  LR: 0.00000000  
Epoch: [5][100/950] Elapsed 0m 28s (remain 3m 55s) Loss: 0.0025(0.0005) Grad: 89604.2422  LR: 0.00000000  
Epoch: [5][120/950] Elapsed 0m 33s (remain 3m 49s) Loss: 0.0001(0.0005) Grad: 15866.7822  LR: 0.00000000  
Epoch: [5][140/950] Elapsed 0m 39s (remain 3m 43s) Loss: 0.0002(0.0004) Grad: 24845.2949  LR: 0.00000000  
Epoch: [5][160/950] Elapsed 0m 44s (remain 3m 37s) Loss: 0.0000(0.0004) Grad: 1944.1212  LR:

Epoch 5 - avg_train_loss: 0.0003  avg_val_loss: 0.1277  time: 283s
INFO:__main__:Epoch 5 - avg_train_loss: 0.0003  avg_val_loss: 0.1277  time: 283s
Epoch 5 - Score: 0.5081  Scores: [0.5456146725600978, 0.5135118831744238, 0.45250204006176176, 0.524141167812546, 0.5256965106765492, 0.4871989665140227]
INFO:__main__:Epoch 5 - Score: 0.5081  Scores: [0.5456146725600978, 0.5135118831744238, 0.45250204006176176, 0.524141167812546, 0.5256965106765492, 0.4871989665140227]


EVAL: [124/125] Elapsed 0m 15s (remain 0m 0s) Loss: 0.0854(0.1277) 


========== fold: 0 result ==========
INFO:__main__:========== fold: 0 result ==========
Score: 0.4744  Scores: [0.4955053859707873, 0.48258737323618073, 0.43359696928816804, 0.4744697259056485, 0.4935083034115342, 0.4669559239043084]
INFO:__main__:Score: 0.4744  Scores: [0.4955053859707873, 0.48258737323618073, 0.43359696928816804, 0.4744697259056485, 0.4935083034115342, 0.4669559239043084]
========== CV ==========
INFO:__main__:========== CV ==========
Score: 0.4744  Scores: [0.4955053859707873, 0.48258737323618073, 0.43359696928816804, 0.4744697259056485, 0.4935083034115342, 0.4669559239043084]
INFO:__main__:Score: 0.4744  Scores: [0.4955053859707873, 0.48258737323618073, 0.43359696928816804, 0.4744697259056485, 0.4935083034115342, 0.4669559239043084]


[fold0] avg_train_loss,█▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold0] avg_val_loss,█▁▂▂▁▁▂▂▂▂▃▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
[fold0] epoch,▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█
[fold0] loss,▅█▂▂▃▄▃▁▂▃▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold0] lr,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
[fold0] score,█▁▂▂▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
[fold0] avg_train_loss,0.00029
[fold0] avg_val_loss,0.12772
[fold0] epoch,5
[fold0] loss,0.0
[fold0] lr,0.0


In [23]:
# if __name__ == '__main__':
    
#     def get_result(oof_df):
#         labels = oof_df[CFG.target_cols].values
#         preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
#         score, scores = get_score(labels, preds)
#         LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    
#     if CFG.train:
#         oof_df = pd.DataFrame()
#         for fold in range(CFG.n_fold):
#             if fold in CFG.trn_fold:
#                 _oof_df = train_loop(train, fold)
#                 oof_df = pd.concat([oof_df, _oof_df])
#                 LOGGER.info(f"========== fold: {fold} result ==========")
#                 get_result(_oof_df)
#         oof_df = oof_df.reset_index(drop=True)
#         LOGGER.info(f"========== CV ==========")
#         get_result(oof_df)
#         oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
#     if CFG.wandb:
#         wandb.finish()